In [1]:
%env CUDA_VISIBLE_DEVICES=3
from pathlib import Path
from pprint import pprint

import pyarrow.dataset as ds

from config import Config

cfg = Config()
# cfg.device="cpu"
pprint(cfg)
encoder_cfg = cfg.encoders[0]

input_path = str(
    Path(cfg.output_dir) / encoder_cfg.name / encoder_cfg.ood_dataset_name
)
dataset = ds.dataset(input_path, format="parquet")
train_set_size = dataset.count_rows()

env: CUDA_VISIBLE_DEVICES=3
Config(device='cuda',
       worker_id=0,
       worker_total=20,
       dry_run=False,
       debug=False,
       output_dir='/raid/pdpl/trak/grads/',
       save_dir='/raid/pdpl/trak/trak_results/',
       write_chunks=1000,
       seed=42,
       proj_dim=2048,
       num_contrastive_samples=50000,
       datasets={'Food101': DatasetConfig(uri='/datasets/food101/shards/food101-train-{000000..000075}.tar',
                                          uris=None,
                                          size=None,
                                          num_workers=16,
                                          splittable=True,
                                          custom=True),
                 'commonpool': DatasetConfig(uri='/datasets/datacomp/shards/{00000000..00001287}.tar',
                                             uris=None,
                                             size=None,
                                             num_workers=16,
     

In [2]:
import numpy as np
from tqdm.rich import tqdm

batch_size = 16384
scanner = dataset.scanner(columns=["grads", "uid"], batch_size=batch_size)
batches = scanner.to_batches()
grads_list = []
uids_list = []
for batch in tqdm(scanner.to_batches(), total=train_set_size // batch_size):
    grads_list.extend(batch.column("grads").to_numpy(zero_copy_only=False))
    uids_list.extend(batch.column("uid").to_numpy(zero_copy_only=False))
g = np.stack(grads_list)
uids = np.stack(uids_list)

Output()

/tmp/ipykernel_112378/130825196.py:9: TqdmExperimentalWarning: rich is experimental/alpha
  for batch in tqdm(scanner.to_batches(), total=train_set_size // batch_size):


In [ ]:
# Sort g and uids arrays by uids
sort_indices = np.argsort(uids)
g = g[sort_indices]
uids = uids[sort_indices]


In [4]:
import torch as ch
from torch import Tensor


def get_xtx(grads: Tensor, batch_size=20_000) -> Tensor:
    proj_dim = grads.shape[1]
    result = ch.zeros(proj_dim, proj_dim, dtype=grads.dtype, device="cuda")
    blocks = ch.split(grads, split_size_or_sections=batch_size, dim=0)

    for block in tqdm(blocks):
        result += block.T.to("cuda") @ block.to("cuda")

    return result

In [5]:
xtx = get_xtx(ch.tensor(g, device="cpu"))

Output()

/tmp/ipykernel_112378/1931852164.py:10: TqdmExperimentalWarning: rich is experimental/alpha
  for block in tqdm(blocks):


In [6]:
def get_x_xtx_inv(
    grads: Tensor, xtx: Tensor, lambda_reg=0.0, batch_size=20_000
) -> Tensor:
    xtx_reg = xtx + lambda_reg * ch.eye(
        xtx.size(dim=0), device=xtx.device, dtype=xtx.dtype
    )
    xtx_inv = ch.linalg.inv(xtx_reg.to(ch.float32))

    # center X^TX inverse a bit to avoid numerical issues when going to float16
    xtx_inv /= xtx_inv.abs().mean()
    xtx_inv = xtx_inv.to(grads.dtype)

    grads_blocks = ch.split(grads, split_size_or_sections=batch_size, dim=0)

    # Move xtx_inv to GPU once before the loop
    xtx_inv_gpu = xtx_inv.cuda()

    # Process blocks on GPU
    result_blocks = []
    for block in tqdm(grads_blocks, desc="Processing blocks"):
        # Move block to GPU, compute, then back to CPU
        block_gpu = block.cuda()
        result_gpu = block_gpu @ xtx_inv_gpu
        result_blocks.append(result_gpu.cpu())

    # Concatenate results on CPU
    result = ch.cat(result_blocks)

    return result.to(dtype=grads.dtype)

In [7]:
x_xtx_inv = get_x_xtx_inv(ch.tensor(g, device="cpu"), xtx)

Output()

/tmp/ipykernel_112378/1614473132.py:20: TqdmExperimentalWarning: rich is experimental/alpha
  for block in tqdm(grads_blocks, desc="Processing blocks"):


In [8]:
features = x_xtx_inv

In [9]:
# get target features
target = "fairvision/amd"
input_path = str(Path(cfg.output_dir) / encoder_cfg.name / target)
dataset_target = ds.dataset(input_path, format="parquet")
batch_size = 16384
scanner = dataset_target.scanner(
    columns=["grads", "uid"], batch_size=batch_size
)
batches = scanner.to_batches()
grads_list = []
uids_list = []
for batch in tqdm(
    scanner.to_batches(), total=dataset_target.count_rows() // batch_size
):
    grads_list.extend(batch.column("grads").to_numpy(zero_copy_only=False))
    uids_list.extend(batch.column("uid").to_numpy(zero_copy_only=False))
g_target = np.stack(grads_list)
uids_target = np.stack(uids_list)

Output()

/tmp/ipykernel_112378/4260970767.py:12: TqdmExperimentalWarning: rich is experimental/alpha
  for batch in tqdm(


In [10]:
from trak.utils import get_matrix_mult_blockwise

In [1]:
full_scores = get_matrix_mult_blockwise(
    features, ch.tensor(g_target, device="cpu"), ch.float16, bs=2048
)


NameError: name 'get_matrix_mult_blockwise' is not defined